In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
urls = [
    'https://www.youtube.com/channel/UC0RhatS1pyxInC00YKjjBqQ'
]

In [7]:
def yt_video_summary_to_df(url:str):
    """Converts YouTube video stats to dataframe
    
    Args:
    url: (str) 
    """
    
    driver = webdriver.Chrome()
    driver.get(f'{url}/videos?view=0&sort=p&flow=grid')
    content = driver.page_source.encode('utf-8').strip()
    soup = BeautifulSoup(content, 'lxml')
    titles_html = soup.findAll('yt-formatted-string', id='video-title')
    views_html = soup.findAll('span', class_='inline-metadata-item style-scope ytd-video-meta-block')
    title, views, date = [], [], []
    
    for i in range(len(titles_html)):
        title.append(titles_html[i].text)
        views.append(views_html[i*2].text)
        date.append(views_html[i*2+1].text)
        
    return pd.DataFrame({
        'title': title,
        'views':views,
        'date':date})

In [8]:
url = 'https://www.youtube.com/channel/UC0RhatS1pyxInC00YKjjBqQ'
geek_df = yt_video_summary_to_df(url)
geek_df.head()

,title,views,date
0,Ace your DevOps interview | GeeksforGeeks,348 views,1 day ago
1,How to Get Free Google Cloud Qwiklabs Credits ...,3.1K views,2 weeks ago
2,"GeeksforGeeks Community – Connect, Ask, Learn ...",722 views,2 weeks ago
3,Become a SDET Expert with GeeksforGeeks,8.1K views,2 weeks ago
4,Geeks Premier League 2023 | India's Biggest Co...,1.5K views,3 weeks ago


In [10]:
magnus_df = yt_video_summary_to_df('https://www.youtube.com/@magmidt')
magnus_df.head()

,title,views,date
0,This is how you train to become the World’s st...,290K views,6 days ago
1,“World’s best climbers can’t do these boulders”,1.3M views,3 weeks ago
2,I'm going back to competing !!!,914K views,4 weeks ago
3,The most dangerous military unit above the Arc...,1.1M views,1 month ago
4,I tried to join the Norwegian Coastal Ranger C...,4.1M views,1 month ago


In [11]:
magnus_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   30 non-null     object
 1   views   30 non-null     object
 2   date    30 non-null     object
dtypes: object(3)
memory usage: 848.0+ bytes


In [12]:
magnus_df

,title,views,date
0,This is how you train to become the World’s st...,290K views,6 days ago
1,“World’s best climbers can’t do these boulders”,1.3M views,3 weeks ago
2,I'm going back to competing !!!,914K views,4 weeks ago
3,The most dangerous military unit above the Arc...,1.1M views,1 month ago
4,I tried to join the Norwegian Coastal Ranger C...,4.1M views,1 month ago
5,Teaching beginner how to go from V0 to V5 in o...,525K views,2 months ago
6,Powerlifters try to complete the most difficul...,1.2M views,3 months ago
7,This 73-Year-Old man has the World’s strongest...,2.6M views,3 months ago
8,bodybuilders try the Rolling Thunder Challenge,532K views,4 months ago
9,why I had to take a long break from climbing,834K views,4 months ago
